In [ ]:
import pandas as pd
import warnings
import numpy as np
import itertools

import statsmodels.api as sm
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')

from sklearn import linear_model

import tensorflow as tf
sess = tf.Session()

# Data Preprocessing

In [ ]:
# Read Power Data
power_path = "../../../data/testing/power.csv" # "power.csv"
power = pd.read_csv(power_path, index_col=[0], parse_dates=True, names=["dt", "power"])

In [ ]:
# Read Ambient Temperature Data
temp_path = "../../../data/testing/amb_temp.csv" # "amb_temp.csv"
temp = pd.read_csv(temp_path, index_col=[0], parse_dates=True, names=["dt", "temp"])

In [ ]:
df = pd.DataFrame()

In [ ]:
df = power.resample('15T').mean()
df["temp"] = temp.resample('15T').mean()

In [ ]:
df.power.plot()

# Learn Physical Model

In [ ]:
# here we have calculated the best possible A and b 
# Hence we will predict the power based on these values now, 
# formula is Power = A* Temp + b

In [ ]:
train_boundry = int(df.shape[0] * 0.6)

findx = df.index.values[0]
mindx = df.index.values[train_boundry]

In [ ]:
# Split the data
X_train = pd.DataFrame(df.ix[findx:mindx, "temp"])
y_train = pd.DataFrame(df.ix[findx:mindx, "power"])

In [ ]:
print X_train.head(), '\n', y_train.head(), X_train.shape, y_train.shape

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

In [ ]:
print regr.coef_, regr.intercept_

In [ ]:
X_test = pd.DataFrame(df.ix[mindx:, "temp"])
y_test = pd.DataFrame(df.ix[mindx:, "power"])

In [ ]:
fig, ax = plt.subplots()

ax.plot(regr.predict(X_test), color='red')
ax.plot(y_test.values, color='blue')

# SARIMA MODEL

In [ ]:
# Split the data
X_Strain = df.ix[findx:mindx, "power"]
X_Stest = df.ix[mindx:, "power"]

In [ ]:
# Train the model
mod = sm.tsa.statespace.SARIMAX(X_Strain, order=(3, 1, 2), seasonal_order=(1, 1, 1, 96))
results = mod.fit()

In [ ]:
results.summary()

# Learn lower and upper thresholds

In [ ]:
# Predict on training dataset
start = X_train.index[0]
end = X_train.index[-1]

df_thresh = pd.DataFrame(results.predict(start, end))
df_thresh.columns = ["pred"]

df_thresh["actual"] = X_train
df_thresh.head()

In [ ]:
fig, ax = plt.subplots()
df_thresh.plot(ax=ax)

In [ ]:
df_thresh["epsilon"] = (df_thresh["actual"] - df_thresh["pred"]) / df_thresh["pred"]

In [ ]:
# H and L for anomaly detection
alpha = 0.998

H = df_thresh.epsilon.quantile(alpha)
L = df_thresh.epsilon.quantile(1.0 - alpha)

print H, L

# Predict Power and Anomaly

In [ ]:
start = X_test.index[0]
end = X_test.index[-1]

print start, end

In [ ]:
df_pred = pd.DataFrame(results.predict(start, end))
df_pred.columns = ["pred"]

df_pred["actual"] = X_test
df_pred.head()

In [ ]:
fig, ax = plt.subplots()
df_pred.plot(ax=ax)

In [ ]:
df_pred["anomaly"] = 0
df_pred.head()

In [ ]:
df_pred.loc[df_pred.epsilon > H, "anomaly"] = 1
df_pred.loc[df_pred.epsilon < L, "anomaly"] = -1

In [ ]:
df_pred[df_pred.epsilon < L]

In [ ]:
df_pred[df_pred.epsilon > H]